In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
print("Memuat data...")
df = pd.read_csv('df_final.csv')

Memuat data...


In [ ]:
df

,Kabupaten/Kota,Tahun,Luas Wilayah,Indeks Pembangunan Manusia (IPM),Harapan Lama Sekolah (Tahun),Rata-rata Lama sekolah (Tahun),Angka Partisipasi Sekolah (APS) Usia 7-12 (Persen),Angka Partisipasi Sekolah (APS) Usia 13-15 (Persen),Angka Partisipasi Sekolah (APS) Usia 16-18 (Persen),Angka Partisipasi Sekolah (APS) Usia 19-23 (Persen),...,Persentase Penduduk (Persen),Persentase Penduduk Miskin (Persen),Tingkat Pengangguran Terbuka (TPT) - Agustus,Tingkat Partisipasi Angkatan Kerja - Agustus,Pengeluaran Per Kapita Riil Disesuaikan (Ribu Rupiah),Persentase Rumah Tangga dengan Akses Air Bersih,Laju Pertumbuhan Penduduk per Tahun (persen),Kepadatan Penduduk per km persegi (km²),Gini Ratio,Cluster
0,Pacitan,2020,1389.92,69.28,12.64,7.60,99.82,97.25,77.61,14.71,...,1.44,14.54,2.28,80.36,8796,84.43,0.78,422.0,0.351,1
1,Ponorogo,2020,1305.70,71.57,13.73,7.54,99.61,98.93,78.66,24.69,...,2.33,9.95,4.45,71.51,9670,91.15,1.01,727.0,0.380,1
2,Trenggalek,2020,1147.22,70.10,12.35,7.55,99.70,94.22,78.70,13.86,...,1.80,11.62,4.11,75.72,9630,80.64,0.78,637.0,0.379,1
3,Tulungagung,2020,1055.65,73.15,13.31,8.33,99.58,98.47,79.72,25.06,...,2.68,7.33,4.61,73.17,10705,96.75,0.93,1032.0,0.336,1
4,Blitar,2020,1336.48,71.02,12.46,7.39,99.39,98.19,70.33,19.61,...,3.01,9.33,3.82,70.83,10654,95.35,0.89,916.0,0.348,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,Kota Pasuruan,2024,35.29,78.30,13.67,9.78,99.94,94.19,83.91,19.99,...,0.52,6.32,5.64,75.65,14250,99.38,1.43,5626.0,0.334,2
186,Kota Mojokerto,2024,20.21,80.90,14.13,11.05,98.97,99.60,87.84,29.55,...,0.33,5.57,4.73,72.50,14422,99.95,0.99,6796.0,0.357,2
187,Kota Madiun,2024,33.92,83.71,14.54,11.82,99.65,99.90,79.96,31.18,...,0.48,4.38,5.85,69.29,17115,99.10,0.89,5585.0,0.435,2
188,Kota Surabaya,2024,350.54,83.99,14.87,10.70,98.81,99.95,70.99,42.46,...,6.99,3.96,6.76,68.73,18977,98.15,0.44,8698.0,0.381,2


In [ ]:
df.columns

Index(['Kabupaten/Kota', 'Tahun', 'Luas Wilayah',
       'Indeks Pembangunan Manusia (IPM)', 'Harapan Lama Sekolah (Tahun)',
       'Rata-rata Lama sekolah (Tahun)',
       'Angka Partisipasi Sekolah (APS) Usia 7-12 (Persen)',
       'Angka Partisipasi Sekolah (APS) Usia 13-15 (Persen)',
       'Angka Partisipasi Sekolah (APS) Usia 16-18 (Persen)',
       'Angka Partisipasi Sekolah (APS) Usia 19-23 (Persen)',
       'Rasio Guru per Siswa', 'Jumlah Penduduk (Ribu)',
       'Persentase Penduduk (Persen)', 'Persentase Penduduk Miskin (Persen)',
       'Tingkat Pengangguran Terbuka (TPT) - Agustus',
       'Tingkat Partisipasi Angkatan Kerja - Agustus',
       'Pengeluaran Per Kapita Riil Disesuaikan (Ribu Rupiah)',
       'Persentase Rumah Tangga dengan Akses Air Bersih',
       'Laju Pertumbuhan Penduduk per Tahun (persen)',
       'Kepadatan Penduduk per km persegi (km²)', 'Gini Ratio', 'Cluster'],
      dtype='object')

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split

def train_and_get_importance_per_cluster(df_path='df_final.csv'):
    print("Memuat data...")
    try:
        df = pd.read_csv(df_path)
    except FileNotFoundError:
        print(f"Error: File '{df_path}' tidak ditemukan.")
        return

    X = df.drop(columns=[
        'Cluster',
        'Kabupaten/Kota',
        'Tahun',
        'Indeks Pembangunan Manusia (IPM)',
        'Kepadatan Penduduk per km persegi (km²)',
        'Persentase Penduduk (Persen)',
        'Pengeluaran Per Kapita Riil Disesuaikan (Ribu Rupiah)',
        'Jumlah Penduduk (Ribu)',
        'Luas Wilayah'
    ])

    unique_clusters = sorted(df['Cluster'].unique())

    print("-" * 50)

    for cluster_label in unique_clusters:
        print(f"\n Menganalisis Model untuk Cluster {cluster_label} (Wilayah vs. Semua Lainnya)")
        print("-" * 50)

        y_binary = df['Cluster'].apply(lambda x: 1 if x == cluster_label else 0)
        X_train, X_test, y_train, y_test = train_test_split(
            X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
        )

        lgbm_binary_model = lgb.LGBMClassifier(objective='binary', random_state=42)
        lgbm_binary_model.fit(X_train, y_train)


        feature_importances = pd.DataFrame({
            'feature': X_train.columns,
            'importance': lgbm_binary_model.feature_importances_
        }).sort_values('importance', ascending=False)

        print(f"10 Fitur Terpenting untuk Mendeteksi Cluster {cluster_label}:")
        print(feature_importances.head(10))

train_and_get_importance_per_cluster()

Memuat data...
--------------------------------------------------

 Menganalisis Model untuk Cluster 0 (Wilayah vs. Semua Lainnya)
--------------------------------------------------
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36, number of negative: 116
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000053 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 626
[LightGBM] [Info] Number of data points in the train set: 152, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.236842 -> initscore=-1.170071
[LightGBM] [Info] Start training from score -1.170071
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

In [ ]:
X_eks = df.drop(columns=[
        'Cluster',
        'Kabupaten/Kota',
        'Tahun',
        'Indeks Pembangunan Manusia (IPM)',
        'Kepadatan Penduduk per km persegi (km²)',
        'Persentase Penduduk (Persen)',
        'Pengeluaran Per Kapita Riil Disesuaikan (Ribu Rupiah)',
        'Jumlah Penduduk (Ribu)',
        'Luas Wilayah'])

X_eks.columns

Index(['Harapan Lama Sekolah (Tahun)', 'Rata-rata Lama sekolah (Tahun)',
       'Angka Partisipasi Sekolah (APS) Usia 7-12 (Persen)',
       'Angka Partisipasi Sekolah (APS) Usia 13-15 (Persen)',
       'Angka Partisipasi Sekolah (APS) Usia 16-18 (Persen)',
       'Angka Partisipasi Sekolah (APS) Usia 19-23 (Persen)',
       'Rasio Guru per Siswa', 'Persentase Penduduk Miskin (Persen)',
       'Tingkat Pengangguran Terbuka (TPT) - Agustus',
       'Tingkat Partisipasi Angkatan Kerja - Agustus',
       'Persentase Rumah Tangga dengan Akses Air Bersih',
       'Laju Pertumbuhan Penduduk per Tahun (persen)', 'Gini Ratio'],
      dtype='object')

In [ ]:
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.inspection import PartialDependenceDisplay
import matplotlib.pyplot as plt

def generate_pdp_per_cluster(df_path='df_final.csv'):
    """
    Fungsi untuk melatih model biner per cluster dan membuat
    Partial Dependence Plot (PDP) untuk fitur-fitur terpentingnya.
    """
    print("Memuat data...")
    try:
        df = pd.read_csv(df_path)
    except FileNotFoundError:
        print(f"Error: File '{df_path}' tidak ditemukan.")
        return

    X = df.drop(columns=[
        'Cluster',
        'Kabupaten/Kota',
        'Tahun',
        'Indeks Pembangunan Manusia (IPM)',
        'Kepadatan Penduduk per km persegi (km²)',
        'Persentase Penduduk (Persen)',
        'Pengeluaran Per Kapita Riil Disesuaikan (Ribu Rupiah)',
        'Jumlah Penduduk (Ribu)',
        'Luas Wilayah'
    ])

    top_features_per_cluster = {
        0: [
            'Rata-rata Lama sekolah (Tahun)', 'Rasio Guru per Siswa',
            'Persentase Rumah Tangga dengan Akses Air Bersih',
            'Tingkat Pengangguran Terbuka (TPT) - Agustus',
            'Laju Pertumbuhan Penduduk per Tahun (persen)'
        ],
        1: [
            'Rata-rata Lama sekolah (Tahun)', 'Rasio Guru per Siswa',
            'Angka Partisipasi Sekolah (APS) Usia 19-23 (Persen)',
            'Gini Ratio', 'Persentase Rumah Tangga dengan Akses Air Bersih'
        ],
        2: [
            'Rata-rata Lama sekolah (Tahun)',
            'Angka Partisipasi Sekolah (APS) Usia 13-15 (Persen)',
            'Persentase Rumah Tangga dengan Akses Air Bersih',
            'Angka Partisipasi Sekolah (APS) Usia 19-23 (Persen)',
            'Tingkat Pengangguran Terbuka (TPT) - Agustus'
        ]
    }

    unique_clusters = sorted(df['Cluster'].unique())
    print("-" * 50)

    for cluster_label in unique_clusters:
        print(f"\nMemproses Model dan PDP untuk Cluster {cluster_label}...")

        y_binary = df['Cluster'].apply(lambda x: 1 if x == cluster_label else 0)
        X_train, _, y_train, _ = train_test_split(
            X, y_binary, test_size=0.2, random_state=42, stratify=y_binary
        )

        lgbm_binary_model = lgb.LGBMClassifier(objective='binary', random_state=42)
        lgbm_binary_model.fit(X_train, y_train)

        features_to_plot = top_features_per_cluster[cluster_label]
        print(f"Membuat PDP untuk fitur: {features_to_plot}")

        fig, ax = plt.subplots(figsize=(15, 10))
        fig.suptitle(f'Partial Dependence Plot untuk Deteksi Cluster {cluster_label}', fontsize=16)

        display = PartialDependenceDisplay.from_estimator(
            lgbm_binary_model,
            X_train,
            features=features_to_plot,
            n_cols=3,
            ax=ax
        )

        plt.subplots_adjust(top=0.9)
        plt.savefig(f'pdp_cluster_{cluster_label}.png', bbox_inches='tight')
        plt.close()

        print(f"Plot 'pdp_cluster_{cluster_label}.png' berhasil disimpan.")
        print("-" * 50)

generate_pdp_per_cluster()

Memuat data...
--------------------------------------------------

Memproses Model dan PDP untuk Cluster 0...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 36, number of negative: 116
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000091 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 626
[LightGBM] [Info] Number of data points in the train set: 152, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.236842 -> initscore=-1.170071
[LightGBM] [Info] Start training from score -1.170071
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 